In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torchvision import transforms
from torchvision import datasets
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageDraw, ImageOps
import tkinter as tk

# Tải MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Xây dựng mô hình CNN cho nhận diện chữ số
class DigitCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(DigitCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Khởi tạo mô hình
model = DigitCNN().to('cpu')

# Tải trọng số đã lưu
model.load_state_dict(torch.load('mnist_digit_cnn_model.pth', map_location='cpu'))
model.eval()

# Tạo cửa sổ ứng dụng để vẽ chữ số
class PaintApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Draw a Digit")
        self.canvas = tk.Canvas(root, width=280, height=280, bg='white')
        self.canvas.pack()
        self.button_predict = tk.Button(root, text="Predict", command=self.predict)
        self.button_predict.pack()
        self.button_clear = tk.Button(root, text="Clear", command=self.clear)
        self.button_clear.pack()
        self.image = Image.new("L", (280, 280), 255)
        self.draw = ImageDraw.Draw(self.image)
        self.canvas.bind("<B1-Motion>", self.paint)
        self.canvas.bind("<ButtonPress-1>", self.paint)

    def paint(self, event):
        x1, y1 = (event.x - 7), (event.y - 7)
        x2, y2 = (event.x + 7), (event.y + 7)
        self.canvas.create_oval(x1, y1, x2, y2, fill='black', width=15)
        self.draw.ellipse([x1, y1, x2, y2], fill=0)

    def clear(self):
        self.canvas.delete("all")
        self.draw.rectangle([0, 0, 280, 280], fill=255)

    def predict(self):
        # Chuyển đổi hình ảnh thành định dạng phù hợp cho mô hình
        image = self.image.resize((28, 28))
        image = self.image.resize((28, 28))
        image = ImageOps.invert(image)
        image = transforms.ToTensor()(image)
        image = transforms.Normalize((0.5,), (0.5,))(image)
        image = image.unsqueeze(0)

        # Dự đoán
        with torch.no_grad():
            output = model(image)
            _, predicted = torch.max(output, 1)
            label = predicted.item()
            result_text = f'Dự đoán nhãn: {label}'
            print(result_text)
            self.root.title(result_text)

# Chạy ứng dụng
root = tk.Tk()
app = PaintApp(root)
root.mainloop()


/var/folders/1n/0p4dx9yd4pjbq77jryb4_7xw0000gn/T/ipykernel_15274/862706630.py:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('mnist_digit

Dự đoán nhãn: 3
Dự đoán nhãn: 1
Dự đoán nhãn: 4
Dự đoán nhãn: 5
Dự đoán nhãn: 5
Dự đoán nhãn: 6
Dự đoán nhãn: 7
Dự đoán nhãn: 3
Dự đoán nhãn: 3
Dự đoán nhãn: 3
Dự đoán nhãn: 3
Dự đoán nhãn: 3
Dự đoán nhãn: 1
Dự đoán nhãn: 2
Dự đoán nhãn: 0
Dự đoán nhãn: 8


In [ ]:
import torch
import torch.nn as nn
import torch.utils.data as data
from torchvision import transforms, datasets
import numpy as np
from PIL import Image, ImageDraw, ImageOps, ImageTk
import tkinter as tk
from tkinter import filedialog

# Tải mô hình flower_cnn_model.pth đã lưu
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# Xây dựng mô hình CNN cho nhận diện tên các loại hoa
class FlowerCNN(nn.Module):
    def __init__(self, num_classes=102):
        super(FlowerCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 128 * 8 * 8)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Khởi tạo mô hình
model = FlowerCNN(num_classes=1020).to('cpu')

# Tải trọng số đã lưu
model.load_state_dict(torch.load('flower_cnn_model.pth', map_location='cpu'))
model.eval()

# Tạo cửa sổ ứng dụng để người dùng tải ảnh vào và dự đoán
class FlowerRecognitionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Flower Recognition App")
        self.label = tk.Label(root, text="Please upload an image of a flower.")
        self.label.pack()
        self.canvas = tk.Canvas(root, width=300, height=300, bg='white')
        self.canvas.pack()
        self.button_upload = tk.Button(root, text="Upload Image", command=self.upload_image)
        self.button_upload.pack()
        self.button_predict = tk.Button(root, text="Predict", command=self.predict)
        self.button_predict.pack()
        self.image = None

    def upload_image(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            self.image = Image.open(file_path).convert('RGB')
            self.image.thumbnail((300, 300))
            self.photo = ImageTk.PhotoImage(self.image)
            self.canvas.create_image(150, 150, image=self.photo)

    def predict(self):
        if self.image is not None:
            # Chuyển đổi hình ảnh thành định dạng phù hợp cho mô hình
            image = self.image.resize((64, 64))
            image = transform(image).unsqueeze(0)

            # Dự đoán
            with torch.no_grad():
                output = model(image)
                _, predicted = torch.max(output, 1)
                label = predicted.item()
                # Lấy tên loại hoa từ danh sách lớp
                class_names = [
                    'Pink Primrose', 'Hard-Leaved Pocket Orchid', 'Canterbury Bells', 'Sweet William',
                    'Tiger Lily', 'Moon Orchid', 'Bird of Paradise', 'Monkshood', 'Globe Thistle',
                    'Snapdragon', 'Colt\'s Foot', 'King Protea', 'Spear Thistle', 'Yellow Iris',
                    'Globe Flower', 'Purple Coneflower', 'Peruvian Lily', 'Balloon Flower', 'Giant White Arum Lily',
                    'Fire Lily', 'Pincushion Flower', 'Fritillary', 'Red Ginger', 'Grape Hyacinth',
                    'Corn Poppy', 'Prince of Wales Feather', 'Stemless Gentian', 'Artichoke',
                    'Sweet Pea', 'Carnation', 'Garden Phlox', 'Love in the Mist', 'Mexican Aster',
                    'Alpine Sea Holly', 'Ruby-Lipped Cattleya', 'Cape Flower', 'Great Masterwort',
                    'Siam Tulip', 'Lenten Rose', 'Barbeton Daisy', 'Daffodil', 'Sword Lily',
                    'Poinsettia', 'Bolero Deep Blue', 'Wallflower', 'Marigold', 'Buttercup', 'Oxeye Daisy',
                    'Common Dandelion', 'Petunia', 'Wild Pansy', 'Primula', 'Sunflower', 'Pelargonium',
                    'Bishop of Llandaff', 'Gaura', 'Geranium', 'Orange Dahlia', 'Pink-Yellow Dahlia',
                    'Cautleya Spicata', 'Japanese Anemone', 'Black-Eyed Susan', 'Silverbush',
                    'Californian Poppy', 'Osteospermum', 'Spring Crocus', 'Bearded Iris', 'Windflower',
                    'Tree Poppy', 'Gazania', 'Azalea', 'Water Lily', 'Rose', 'Thorn Apple', 'Morning Glory',
                    'Passion Flower', 'Lotus', 'Toad Lily', 'Anthurium', 'Frangipani', 'Clematis',
                    'Hibiscus', 'Columbine', 'Desert-rose', 'Tree Mallow', 'Magnolia', 'Cyclamen ',
                    'Watercress', 'Canna Lily', 'Hippeastrum ', 'Bee Balm', 'Pink Quill',
                    'Foxglove', 'Bougainvillea', 'Camellia', 'Mallow', 'Mexican Petunia', 'Bromelia',
                    'Blanket Flower', 'Trumpet Creeper', 'Blackberry Lily'
                ]
                flower_name = class_names[label]
                result_text = f'Dự đoán loại hoa: {flower_name}'
                print(result_text)
                self.root.title(result_text)
        else:
            self.label.config(text="Please upload an image first.")

# Chạy ứng dụng
root = tk.Tk()
app = FlowerRecognitionApp(root)
root.mainloop()